In [1]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

In [2]:
from astropy.utils import iers
iers.conf.auto_download = False

In [3]:
d = salobj.Domain()

In [4]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [5]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

[None, None, None, None, None, None, None]

AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 10 messages


# ATMCS

In [8]:
await asyncio.sleep(5.)
print("Done")
await salobj.set_summary_state(atmcs, salobj.State.ENABLED)

RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 31 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 31 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 31 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 31 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 31 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 25 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 31 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 25 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 31 messages
RemoteEvent(ATHexapod, 0, he

Done


[<State.DISABLED: 1>, <State.ENABLED: 2>]

In [ ]:
await salobj.set_summary_state(atmcs, salobj.State.STANDBY)

In [7]:
await asyncio.sleep(5.)
print("Done")

await asyncio.gather(salobj.set_summary_state(atmcs, salobj.State.ENABLED),
                     salobj.set_summary_state(atptg, salobj.State.ENABLED))

RemoteTelemetry(ATDome, 1, position) falling behind; read 79 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 79 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 16 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 16 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 16 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 16 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 16 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 16 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 16 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 15 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 12 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 16 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 12 messages
RemoteTelemetry(ATPneumatics, 0, 

Done


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1523520179, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='ERROR: Command standby rejected while in StandbyState state.')

# ATPtg

In [ ]:
await salobj.set_summary_state(atptg, salobj.State.STANDBY)

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [ ]:
await asyncio.sleep(5.)
print("Done")
await atptg.tel_timeAndDate.next(flush=True, timeout=5)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

In [ ]:
await atptg.cmd_azElTarget.set_start(azDegs=0, elDegs=80.)

In [9]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [10]:
ra = Angle("00 17 58.8690394826", unit=u.hour)
dec = Angle("-13 27 20.308710371", unit=u.deg)
target_name="HD 1388"
radec = ICRS(ra, dec)

In [27]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
RemoteEvent(GenericCamera, 1, heartbeat) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
Rem

58739.218198115756
slew...


RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 100 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 100 messages
RemoteEvent(ATPtg, 0, weatherDataApplied) falling behind; read 32 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 100 messages
RemoteEvent(ATPtg, 0, mountDataWarning) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, azimuthInPosition) falling behind; read 20 messages
RemoteEvent(ATMCS, 0, allAxesInPosition) falling behind; read 20 messages


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [14]:
await atptg.cmd_offsetAzEl.set_start(el=-8., 
                                     az=-25., 
                                         num=0)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 22 elements
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 11 messages


falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements


In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-20., 
                                     az=40., 
                                         num=0)

In [15]:
await atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await atptg.cmd_pointCloseFile.start()

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 35 messages
falling behind; queue contains 15 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 35 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 35 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling beh

In [ ]:
await asyncio.gather(atmcs.close(), 
                     atptg.close(),
                     ataos.close(),
                     atpne.close(),
                     athex.close(),
                     atdome.close(),
                     atdomtraj.close())

In [ ]:
20*360/24., 22*360/24.

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [ ]:
await atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await atptg.cmd_pointCloseFile.start()

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.35, u=0.35, v=0.22)

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.20, u=0.35, v=0.22)

In [18]:
cam = salobj.Remote(d, "GenericCamera", 1)

In [19]:
await cam.start_task

AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 100 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 100 messages
RemoteEvent

In [20]:
import numpy as np

In [21]:
z_seq = np.linspace(0.1, 0.35, 6)
print(z_seq)

[0.1  0.15 0.2  0.25 0.3  0.35]


In [ ]:
z_seq[3:]

In [17]:
for z in z_seq:
    print(f"Positioning hexapod @ z={z:4.2f}")
    await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=z, u=0.35, v=0.22)

    print(f"Taking images: {target_name}_z{z:4.2f}_10s")
    await cam.cmd_takeImages.set_start(numImages=3,
                                     expTime=10.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name}_z{z:4.2f}_10s'
                                    )

Positioning hexapod @ z=0.10
Taking images: 'HD 1388_z0.10_10s
Positioning hexapod @ z=0.15
Taking images: 'HD 1388_z0.15_10s
Positioning hexapod @ z=0.20
Taking images: 'HD 1388_z0.20_10s
Positioning hexapod @ z=0.25
Taking images: 'HD 1388_z0.25_10s
Positioning hexapod @ z=0.30
Taking images: 'HD 1388_z0.30_10s
Positioning hexapod @ z=0.35
Taking images: 'HD 1388_z0.35_10s


In [ ]:
1*360/24.

In [26]:
await cam.cmd_takeImages.set_start(numImages=1,
                                     expTime=5.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name.replace(" ","_")}_test_5s'
                                    )

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 37 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 37 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 17 messages
falling behind; queue contains 16 elements
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 37 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 37 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 37 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 12 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue